In [ ]:
import os, time
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding

def aes_env():
    '''
    Session is a random 32 byte number
    Sequence starts off as a 
    
    '''
    session = os.urandom(26)
    sequence = 10000
    seq_bytes = sequence.to_bytes(2, byteorder='big')
    time_stmp = int(time.time())
    time_bytes = time_stmp.to_bytes(4, 'big')
    session_key = session + seq_bytes + time_bytes
    # print(len(session_key))

    with open("server_pubkey", "rb") as f:
        pem_pub = f.read()
    public_key = serialization.load_pem_public_key(pem_pub)

    with open("client_prikey", "rb") as e:
        client_private_key = serialization.load_pem_private_key(
            e.read(),
            password=None
        )
    
    ciphertext = public_key.encrypt(
        session_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    signed_session = client_private_key.sign(
        ciphertext,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    return ciphertext, signed_session



x, y = aes_env()
print(f"Encrypted message: {x}")
print(f"Signed message: {y}")

32
Encrypted message: b'V\xa8\xd3\xb5\x03\x8e\xc2\x8c\tF-\xe1\xfbd\xb4\x7f\x88\xdd\xf8Z\x95\xf7\x9d[I\xd2ZVS+&D_\xc5;\\GV4\xd0\x7f[O\xfa\x1e{\xdfO\x15\x84ZQ\x8d\x82@\xca\x07k\x8a\xf8\r\xd3B\x1a\x08Z8\xfbG\x10\x10\xdb\xb6\x8fX\x80=\x86V\xd4[K<T\x85\x11\xf6#\xa3X@lk-\x96a\x8c6zJ\x1d\xcb\xd4\x9dc1\xd8\xb4\x16V\x10\x1f\x15\x1d\xb1m\xb2\xcf\x02\xffTV\xf6W\x9f\xd2\xac\x0cp\xf0$mq\xf1\xa1\xd8\x9e\xbf\xf1\xc0\x93e\xf3/0ao(<\x91\x0f\xf6G\xb6\xf8\x88\xcb/2>\xe8\x14\xf0\xbdlA\xf8\xe2)6\xce\xc7U\x06\xbf\xdd\x8e\x08\x9c\x12Q\xd3C\xce\x1c$\xd8\xc9y\xeb\xfd\xb7zZ\xf1V\x82\xecj\xbeF<\x94\x15\xb8,n\xc2\xfeJ\x02|\x12\x9a\xa4\xab\xea\xdciX#\x0e`:\xfa M\xca\x97d\x08\x0f\xd9\xf4\xf9&tH\xce\xb3\x14\xf9XH\xcaL\x08\xfd\xb7\xe3\xa7\x7f\xe9\x9b\xe5]'
Signed message: b'I\xbd\xa3\xe3\xc3\xca\x0f\x8d\x87e\x17\t\x0c\xbf\x1d\xe0:\xc4\xeb\xe7\xac\x84Kk\x91\xfa\x9c\xc05\xe8S{\xe2\xa5\xeaA\xa4~\xe3\x0e\xea\x04\xa0p\\2\xf1\xab\x94\xf5vFia\xbdMM\xd4u\x8d#\xd7\x04\n\xf3\xbb\x07\xd9Z\xf7\x8c\x13S\xb5\x19\xd4`\xb2\x93\xeb[\

## Current Work
- Currently I generate a session key from a 26 byte random number, then add 2 byte sequence number and 4 byte time. Makes a 32 byte session key, I then encrypt it and sign the encryption. I can send all of this to the server.
- Server todo's
  - Take message, check signature 
  - Once confirmed parse out session key
  - If time is within 10 seconds good
  - Add one to sequence
  - I probably need to keep squence out of the session key but I can add it to the message so it can properly be tracked.
  - Then keep working towards how the message is read and decrypted.